In [1]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

In [2]:
from datasets import load_dataset, load_metric

In [4]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Пример документа:

In [5]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

Посмотрим, какие сущности есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type

In [6]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [7]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Напишем функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд. 

In [65]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)
    # print(f"char2word ->  {char2word}")    

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        # print(f"e_  -> {e}")
        # print(f"e_words ->  {e_words}")
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [66]:
print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [10]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=1)

Пример данных

In [11]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
4065,"[Когда, он, у, меня, появился, впервые, ,, я, зашла, в, аптеку, и, попросила, какую-нибудь, мазь, ,, мне, фармацевт, предложила, Ацикловир, ,, но, разных, производителей, :, отечественный, за, 20, рублей, ,, белорусский, за, 45, и, немецкий, за, 150, рублей, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Drugform, O, O, O, O, B-Drugname, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3891,"[После, курса, лечения, ребенок, стал, более, любознательным, ,, но, не, уверенна, ,, что, именно, от, кортексина, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Drugname, O]"
1584,"[После, приема, препарата, ,, покрылась, испариной, ,, наступила, слабость, и, осознание, того, ,, что, пора, уже, к, врачу, .]","[O, O, O, O, B-ADR, I-ADR, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O]"


Соберём все виды меток в список. 

In [12]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Сложим наши данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [13]:
from datasets import Dataset, DatasetDict

In [14]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 4328
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 481
    })
})

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You can directly call this tokenizer on one sentence:

In [16]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [17]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [18]:
example = ner_train[5]
print(example["tokens"])

['Этим', 'средством', 'является', '"', 'Амизон', '"', ',', 'которое', 'хорошо', 'прорекламировано', ',', 'однако', 'такого', 'эффекта', 'как', 'говорят', 'нет', '.']


In [19]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Эти', '##м', 'средство', '##м', 'является', '"', 'А', '##ми', '##зон', '"', ',', 'которое', 'хорошо', 'про', '##рек', '##лами', '##ровано', ',', 'однако', 'такого', 'э', '##ффект', '##а', 'как', 'говорят', 'нет', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [20]:
len(example["tags"]), len(tokenized_input["input_ids"])

(18, 29)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [21]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, 9, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [22]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

29 29


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the flag `label_all_tokens`.

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [23]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [24]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 3130, 3374, 23324, 871, 314, 1556, 14068, 16902, 1029, 6899, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [25]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4328 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [26]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [27]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [28]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [29]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [31]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [32]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

d:\virt_dev\.venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\virt_dev\.venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\virt_dev\.venv\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\virt_dev\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [33]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [35]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 2.711977958679199,
 'eval_precision': 0.017055378608856468,
 'eval_recall': 0.10109622411693057,
 'eval_f1': 0.02918681318681319,
 'eval_accuracy': 0.040036776997659645,
 'eval_runtime': 19.4894,
 'eval_samples_per_second': 24.68,
 'eval_steps_per_second': 1.591}

В начале обучения заморозим все параметры в модели, кроме последнего слоя, и посмотрим, насколько хорошо она обучится.

In [36]:
for param in model.bert.parameters():
    param.requires_grad = False

In [37]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[-0.0243, -0.0206,  0.0011,  ..., -0.0372, -0.0032, -0.0228],
        [ 0.0034,  0.0116,  0.0213,  ..., -0.0061, -0.0106,  0.0349],
        [-0.0100, -0.0223,  0.0108,  ..., -0.0101,  0.0382,  0.0049],
        ...,
        [ 0.0035, -0.0439,  0.0345,  ..., -0.0139, -0.0092, -0.0140],
        [ 0.0041,  0.0031, -0.0408,  ...,  0.0104,  0.0201,  0.0143],
        [-0.0011, -0.0053,  0.0130,  ..., -0.0041, -0.0245, -0.0002]],
       requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)


We can now finetune our model by just calling the `train` method:

In [38]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [39]:
trainer.train()

  0%|          | 0/2710 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 2.0763051509857178, 'eval_precision': 0.028388533259114945, 'eval_recall': 0.06211936662606577, 'eval_f1': 0.03896848137535817, 'eval_accuracy': 0.582246740220662, 'eval_runtime': 13.161, 'eval_samples_per_second': 36.547, 'eval_steps_per_second': 2.355, 'epoch': 1.0}
{'loss': 2.1334, 'learning_rate': 1.6309963099630997e-05, 'epoch': 1.85}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 1.5918083190917969, 'eval_precision': 0.03317535545023697, 'eval_recall': 0.004263093788063338, 'eval_f1': 0.007555315704263358, 'eval_accuracy': 0.8060013373453695, 'eval_runtime': 34.8984, 'eval_samples_per_second': 13.783, 'eval_steps_per_second': 0.888, 'epoch': 2.0}


...

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.8174466490745544, 'eval_precision': 1.0, 'eval_recall': 0.0006090133982947625, 'eval_f1': 0.0012172854534388312, 'eval_accuracy': 0.8184553660982948, 'eval_runtime': 9.947, 'eval_samples_per_second': 48.356, 'eval_steps_per_second': 3.117, 'epoch': 9.0}
{'loss': 0.766, 'learning_rate': 1.5498154981549817e-06, 'epoch': 9.23}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.8144131302833557, 'eval_precision': 1.0, 'eval_recall': 0.0006090133982947625, 'eval_f1': 0.0012172854534388312, 'eval_accuracy': 0.8184553660982948, 'eval_runtime': 9.7809, 'eval_samples_per_second': 49.178, 'eval_steps_per_second': 3.169, 'epoch': 10.0}
{'train_runtime': 1549.621, 'train_samples_per_second': 27.929, 'train_steps_per_second': 1.749, 'train_loss': 1.1588752211679831, 'epoch': 10.0}


TrainOutput(global_step=2710, training_loss=1.1588752211679831, metrics={'train_runtime': 1549.621, 'train_samples_per_second': 27.929, 'train_steps_per_second': 1.749, 'train_loss': 1.1588752211679831, 'epoch': 10.0})

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморозим их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [40]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [41]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [42]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

  0%|          | 0/5420 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.6113221049308777, 'eval_precision': 0.6165540540540541, 'eval_recall': 0.2222898903775883, 'eval_f1': 0.3267681289167413, 'eval_accuracy': 0.8446171848879973, 'eval_runtime': 15.8431, 'eval_samples_per_second': 30.36, 'eval_steps_per_second': 1.957, 'epoch': 1.0}
{'loss': 0.5684, 'learning_rate': 9.07749077490775e-06, 'epoch': 1.85}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.540976881980896, 'eval_precision': 0.5584944048830112, 'eval_recall': 0.3343483556638246, 'eval_f1': 0.41828571428571426, 'eval_accuracy': 0.855650284185891, 'eval_runtime': 15.4965, 'eval_samples_per_second': 31.039, 'eval_steps_per_second': 2.0, 'epoch': 2.0}


...

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.3530827462673187, 'eval_precision': 0.623642943305187, 'eval_recall': 0.6297198538367844, 'eval_f1': 0.6266666666666667, 'eval_accuracy': 0.8988632564359746, 'eval_runtime': 17.3805, 'eval_samples_per_second': 27.675, 'eval_steps_per_second': 1.784, 'epoch': 19.0}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.3528536856174469, 'eval_precision': 0.6253041362530414, 'eval_recall': 0.6260657734470159, 'eval_f1': 0.6256847230675594, 'eval_accuracy': 0.8987796723503845, 'eval_runtime': 17.3232, 'eval_samples_per_second': 27.766, 'eval_steps_per_second': 1.79, 'epoch': 20.0}
{'train_runtime': 8137.7145, 'train_samples_per_second': 10.637, 'train_steps_per_second': 0.666, 'train_loss': 0.34129813781963503, 'epoch': 20.0}


TrainOutput(global_step=5420, training_loss=0.34129813781963503, metrics={'train_runtime': 8137.7145, 'train_samples_per_second': 10.637, 'train_steps_per_second': 0.666, 'train_loss': 0.34129813781963503, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [44]:
trainer.evaluate()

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.3528536856174469,
 'eval_precision': 0.6253041362530414,
 'eval_recall': 0.6260657734470159,
 'eval_f1': 0.6256847230675594,
 'eval_accuracy': 0.8987796723503845,
 'eval_runtime': 24.134,
 'eval_samples_per_second': 19.93,
 'eval_steps_per_second': 1.284,
 'epoch': 20.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [45]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/31 [00:00<?, ?it/s]

{'ADR': {'precision': 0.35233160621761656,
  'recall': 0.25660377358490566,
  'f1': 0.29694323144104806,
  'number': 265},
 'DI': {'precision': 0.4288194444444444,
  'recall': 0.5416666666666666,
  'f1': 0.4786821705426356,
  'number': 456},
 'Drugclass': {'precision': 0.8354430379746836,
  'recall': 0.8198757763975155,
  'f1': 0.8275862068965517,
  'number': 161},
 'Drugform': {'precision': 0.8390804597701149,
  'recall': 0.8202247191011236,
  'f1': 0.8295454545454546,
  'number': 267},
 'Drugname': {'precision': 0.7956043956043956,
  'recall': 0.9027431421446384,
  'f1': 0.8457943925233645,
  'number': 401},
 'Finding': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 92},
 'overall_precision': 0.6253041362530414,
 'overall_recall': 0.6260657734470159,
 'overall_f1': 0.6256847230675594,
 'overall_accuracy': 0.8987796723503845}

In [46]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [47]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,9622,12,71,15,22,33,0,3,13,0,0,0,0
B-ADR,89,78,89,1,0,0,1,3,4,0,0,0,0
B-DI,153,19,265,0,10,4,0,1,4,0,0,0,0
B-Drugclass,18,0,8,132,0,3,0,0,0,0,0,0,0
B-Drugform,30,0,5,1,220,11,0,0,0,0,0,0,0
B-Drugname,18,0,6,2,3,372,0,0,0,0,0,0,0
B-Finding,23,28,33,0,4,3,0,0,1,0,0,0,0
I-ADR,110,25,19,0,1,0,0,24,23,0,0,0,0
I-DI,150,15,42,0,0,1,0,4,40,0,0,0,0
I-Drugclass,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin\\tokenizer_config.json',
 'ner_bert.bin\\special_tokens_map.json',
 'ner_bert.bin\\vocab.txt',
 'ner_bert.bin\\added_tokens.json',
 'ner_bert.bin\\tokenizer.json')

In [49]:
import torch

In [50]:
text = ' '.join(ner_train[8]['tokens'])
text = ' '.join(ner_test[4]['tokens'])
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [52]:
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 29, 13])

In [53]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
О               O         
##хо            O         
##тно           O         
при             O         
##мен           O         
##я             O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
нас             B-DI      
##мор           B-DI      
##ком           B-DI      
,               O         
что             O         
в               O         
м               O         
##ое            O         
##м             O         
случае          O         
я               O         
##вление        O         
очень           O         
часто           O         
##е             O         
.               O         
[SEP]           O         
